In [1]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer
from datetime import datetime


In [17]:
def date_to_julian(date_obj):
    """
    Convert a Python date/datetime to AS400 Julian format (YYDDD).

    Format: YYDDD where YY=year, DDD=day of year
    Example: January 1, 2026 -> 26001

    Args:
        date_obj: datetime.date, datetime.datetime, or string in 'YYYY-MM-DD' format

    Returns:
        int: Julian date in YYDDD format
    """
    if isinstance(date_obj, str):
        date_obj = datetime.strptime(date_obj, '%Y-%m-%d').date()
    elif isinstance(date_obj, datetime):
        date_obj = date_obj.date()

    year = date_obj.year
    year_digits = year % 100
    day_of_year = date_obj.timetuple().tm_yday

    julian = year_digits * 1000 + day_of_year
    return julian

In [3]:
# Define Database Connection for PROD

CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

In [4]:
# Define Database Connection for ccsdta

CONNAS400_CCSDTA = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
"""


In [5]:
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
conn_sql = engine.connect()

In [26]:
SCRAP_COLUMNS = ["ITMID", "ITMDESC", "PRODCODE", "INVCLASS", "TXNCD", "REASON", "PLT", "TXNQTY", "UOM", "FRZSTDCST",
                 "TNREF1", "TXNSDT", "USERID"]
SCRAP_DTYPE = {
    "ITMID": str,
    "ITMDESC": str,
    "PRODCODE": str,
    "INVCLASS": str,
    "TXNCD": str,
    "REASON": str,
    "PLT": str,
    "TXNQTY": float,
    "UOM": str,
    "FRZSTDCST": float,
    "TNREF1": str,
    "TXNSDT": float,
    "USERID": str
}

In [29]:
SCRAP_START = date_to_julian('2026-01-01')
SCRAP_STOP = date_to_julian(datetime.today().strftime('%Y-%m-%d'))
sql_inv = """
    SELECT
    b.plt,
    y.itmid,
    b.qty,
    y.itmdesc,
    SUBSTR (Altdesc, 15, 1) Class
FROM
    CCSDTA.DCSCIM y,
    CCSDTA.DMFCMAR x,
    CCSDTA.DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND b.plt = '09'
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')
"""
sql_scrap = f"""
SELECT
        x.itmid,
        y.itmdesc,
        substr(z.altdesc,12,1) PRODCODE,
        substr(z.altdesc,15,1) INVCLASS,
        txncd,
        reason,
        x.plt,
        txnqty,
        x.uom,
        y.frzstdcst,
        tnref1,
        txnsdt,
        userid
FROM
        CCSDTA.DCSHST x,
        CCSDTA.DCSDIM y,
        CCSDTA.DCSCIM z
WHERE
        x.plt = y.plt
        AND x.itmid = y.itmid
        AND x.itmid = z.itmid
        AND txnsdt > {SCRAP_START}
        AND txnsdt < {SCRAP_STOP}
        AND y.plt = '09'
        AND txncd in ('12', '13')
"""

In [9]:

def pull_data(conn,qry):
    # Connection with error handling and connection management
    result = []
    start = 0
    msg = ""  # Initialize msg
    dbcnxn = None  # Initialize dbcnxn
    try:
        dbcnxn = pyodbc.connect(conn, timeout=30)
        cursor = dbcnxn.cursor()
        start = timer()

        try:
            cursor.execute(qry)
            result = cursor.fetchall()
            msg = str(len(result)) + " Records Processed From Table"
        except Exception as e:
            msg = conn + ' Query Failed: ' + str(e)
            print(msg)
    except pyodbc.Error as e:
        msg = conn + ' Connection Failed: ' + str(e)
        print(msg)
    finally:
        if msg:
            print(msg)
        if dbcnxn is not None:
            try:
                print("Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")
                dbcnxn.close()
            except:
                pass  # Handle case where connection wasn't established

    return result

In [52]:
print(date_to_julian('2026-12-01'))

26335


In [10]:
def _build_scrap_dataframe(raw_records: list) -> pd.DataFrame:
    """
    Build the components inventory DataFrame from raw AS400 records.

    This function is responsible only for shaping, cleaning, and casting
    the data into the expected schema.
    """
    if not raw_records:
        # Return an empty DataFrame with the correct schema
        return pd.DataFrame(columns=SCRAP_COLUMNS)

    df_scrap = pd.DataFrame.from_records(raw_records, columns=SCRAP_COLUMNS)

    # Drop any rows missing required fields before numeric conversion
    df_scrap = df_scrap.dropna(subset=SCRAP_COLUMNS)

    # Convert QTY to numeric, coercing invalid entries to NaN
    df_scrap["TXNQTY"] = pd.to_numeric(df_scrap["TXNQTY"], errors="coerce")

    # Remove rows where QTY could not be converted
    df_scrap = df_scrap.dropna(subset=["TXNQTY"])

    try:
        df_scrap = df_scrap.astype(SCRAP_DTYPE)
    except (TypeError, ValueError) as exc:
        print(f"Error converting component inventory data types: {exc}")
        return pd.DataFrame(columns=SCRAP_COLUMNS)

    # Let pandas choose the most appropriate dtypes (nullable ints, etc.)
    df_scrap = df_scrap.convert_dtypes()

    print(f"Processed {len(df_scrap)} component inventory records")
    return df_scrap

In [27]:
def scrap_tbl(df_data):
    # Build Components Table
    print('Build Scrap SQL Table')
    if df_data.empty:
        print("Warning: Empty DataFrame, skipping SQL insert")
        return

    # Validate required columns exist
    required_columns = SCRAP_COLUMNS
    missing_columns = [col for col in required_columns if col not in df_data.columns]
    if missing_columns:
        error_msg = f"Missing required columns: {missing_columns}"
        print(f"Error: {error_msg}")
        raise ValueError(error_msg)

    data_type_dict = {
        "ITMID": sqlalchemy.types.VARCHAR(255),
        "ITMDESC": sqlalchemy.types.VARCHAR(255),
        "PRODCODE": sqlalchemy.types.VARCHAR(255),
        "INVCLASS": sqlalchemy.types.VARCHAR(255),
        "TXNCD": sqlalchemy.types.VARCHAR(255),
        "REASON": sqlalchemy.types.VARCHAR(255),
        "PLT": sqlalchemy.types.VARCHAR(255),
        "TXNQTY": sqlalchemy.types.FLOAT,
        "FRZSTDCST": sqlalchemy.types.FLOAT,
        "TNREF1": sqlalchemy.types.VARCHAR(255),
        "TXNSDT": sqlalchemy.types.FLOAT,
        "UOM": sqlalchemy.types.VARCHAR(255),
        "USERID": sqlalchemy.types.VARCHAR(255)
    }
    try:
        df_data.to_sql('tblScrapAll', engine, schema='eng', if_exists='replace', index=False,
                         dtype=data_type_dict)
        print(f"Successfully inserted {len(df_data)} records into tblScrapAll")
    except Exception as e:
        print(f"Error inserting data into tblScrapAll: {e}")

In [30]:

data_set = pull_data(CONNAS400_CCSDTA,sql_scrap)
df_scrap1 = _build_scrap_dataframe(data_set)
scrap_tbl(df_scrap1)

5167 Records Processed From Table
Connect/Query Time = 0.393 sec
Processed 5167 component inventory records
Build Scrap SQL Table
Successfully inserted 5167 records into tblScrapAll


#### Build Dataframe for Component Inventory

In [21]:
df_inv = pd.DataFrame.from_records(result)
data_type_dict = {'PLANT': str, 'ITMID': str, 'QTY': int, 'ITMDESC': str , 'CLASS': str}
df_inv.columns = ['PLANT', 'ITMID', 'QTY', 'ITMDESC', 'CLASS']
df_inv = df_inv.dropna()
df_inv = df_inv.astype(data_type_dict)
df_inv = df_inv.convert_dtypes()
df_inv.sample(20)

,PLANT,ITMID,QTY,ITMDESC,CLASS
8943,09,9009051,1350,BF LOX-ON,2
7474,09,6712595,22220,"67-12-5 1/2"" IND LAB HOME DEP @220",5
484,09,M647248206,1800,BULK @100,5
11799,09,676006175,450,FRS 561290 BULK @500,5
6483,09,6336199,962,"CLP - 9/16"" 63-36-1 - 301 301 204 - 5/16",4
12126,09,533040090016,20,VELVAC #022548 9MM W4 NP CLAMP 10PK@100,5
7265,09,67004010405K,100,670040-104 1/2 KIMBALL/MID#50499 10pk MX,5
4425,09,534050080052,1,12MM W5 IDEAL NON-PERF CLP BULK,5
6166,09,6260451,1000,5/16 MC 301-301-302 SZ 004 0.22-0.63,5
6501,09,6388070,96,63-88-0 9/16 WW GRAINGER 5CZC1 (MX),5


In [40]:
def get_inv():
    try:
        dbcnxn1 = pyodbc.connect(CONNAS400_CCSDTA, timeout=30)
        cursor1 = dbcnxn1.cursor()
        start = timer()

        try:
            cursor1.execute(sql_inv)
            result = cursor1.fetchall()
            msg = str(len(result)) + " AS400 Inventory Records Processed From Table"
        except Exception as e:
            msg = 'AS400 Inventory Query Failed: ' + str(e)
            result = []
            print(msg)
        finally:
            print(msg)
            print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")
            return result
    except pyodbc.Error as e:
        msg = 'AS400 Connection Failed: ' + str(e)
        result = []
        print(msg)
    finally:
        if 'dbcnxn1' in locals():
            dbcnxn1.close()

In [41]:
print(get_inv())

AS400 Inventory Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Inventory Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Connect/Query Time = 0.003 sec
[]


In [22]:
def get_emps():
    """Get Employees From iSeries AS400"""
    try:
        dbcnxn = pyodbc.connect(CONNAS400)
        cursor = dbcnxn.cursor()
    except pyodbc.Error as e:
        msg = 'AS400 Connection Failed: ' + str(e)
        print(msg)

    eng_login = ['9999', 'ELMER J FUDD', 'ENG']
    lead_login = ['1208', 'Wile E Coyote', 'ENG']
    eng = ['1208', '9107', '1656', '1472', '1626', '1351']

    strsql = """SELECT STRIP(EMP_CLOCK_NUMBER) As Clock,
                    CONCAT(CONCAT(STRIP(EMP_FIRST_NAME), ' '),
                    STRIP(EMP_LAST_NAME)) As Name,
                    STRIP(EMP_POSITION_CODE) As Code
            FROM PROD.FPCLCKPAY
            WHERE (EMP_LOCATION = 09) AND (EMP_LAST_NAME <> 'TEMP') AND (EMP_SHIFT_TYPE = 'A')
            ORDER BY EMP_CLOCK_NUMBER"""
    try:
        cursor.execute(strsql)
        result = cursor.fetchall()
    except Exception as e:
        msg = 'AS400 Employee Query Failed: ' + str(e)
        result = []
        print(msg)
    else:
        msg = str(len(result)) + " AS400 Employee Records Processed From Table"
        print(msg)
    result.append(eng_login)
    result.append(lead_login)
    for row in result:
        if row[0] in eng:
            row[2] = 'ENG'
    dbcnxn.close()
    return result

In [23]:
print(get_emps())


471 AS400 Employee Records Processed From Table
[('1001', 'DEBBIE POMEROY-GAMMONS', 'DEF'), ('1013', 'BETH THOMPSON', 'DEF'), ('1032', 'THOMAS BOGLE', 'DEF'), ('1039', 'VICKIE ANDERSON', 'DEF'), ('1090', 'BILLY NIPPERS', 'DEF'), ('1122', 'SHELLEY HOUSTON', 'DEF'), ('1139', 'ANTIONETTE BASS', 'DEF'), ('1175', 'WYOMIA CLARK', 'DEF'), ('1176', 'NALEE NHOTSAVANG', 'DEF'), ('1177', 'CHARLES TOOMBS', 'DEF'), ('1213', 'DOUGLAS SHIPP', 'DEF'), ('1238', 'MARICHU RIVERA', 'DEF'), ('1258', 'PHOUVEUNE LAMNGEUN', 'DEF'), ('1318', 'TERESA HOLLIS', 'DEF'), ('1331', 'NANCY CLARK', 'DEF'), ('1332', 'BRENDA CUNNINGHAM', 'DEF'), ('1347', 'HERBERT GRAHAM', 'DEF'), ('1352', 'DALE GERHART', 'DEF'), ('1372', 'SITHONH BOUAPHACHANH', 'DEF'), ('1429', 'GARY BELCHER', 'DEF'), ('1444', 'RILEY BREWER  JR.', 'DEF'), ('1464', 'CHRISTOPHER HIX', 'DEF'), ('1483', 'BARRY KIRBY', 'DEF'), ('1496', 'TERRY MARTIN', 'DEF'), ('1499', 'ACKHASONE PHILAVARN', 'DEF'), ('1514', 'KHAMIN VONGPANYA', 'DEF'), ('1515', 'PHILLIP STINSO